In [9]:
from preprocess import proc
import freq
from freq import read_kw, word_list2freq_dict
from importlib import reload
import pandas as pd
reload(freq)

kw = read_kw()

train = pd.read_csv("./train.csv")[['target', 'comment_text']]
train['toxic'] = 1 * (train['target'] > 0.5)

train_toxic = train[train['toxic'] == 1]
train_not_toxic = train[train['toxic'] == 0].sample(len(train_toxic))
train = pd.concat([train_toxic, train_not_toxic]).sample(len(train_toxic) * 2)

def comment_text_to_vec(comment_text):
    p = proc(comment_text, [1, 2])
    return word_list2freq_dict(kw, p)

train["comment"] = train['comment_text'].apply(comment_text_to_vec)

In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train["comment"], train["toxic"])

import numpy as np
X_train = np.stack(X_train)
X_test = np.stack(X_test)

In [ ]:
m1 = LinearDiscriminantAnalysis()
m1.fit(X_train, y_train)
pred = m1.predict(X_test)

In [11]:
m1.score(X_train, y_train)

0.7488151658767772

In [12]:
m1.score(X_test, y_test)

0.6824644549763034

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score
print(confusion_matrix(pred, y_test))
print(classification_report(y_test, pred, digits=3))

In [13]:
m2 = QuadraticDiscriminantAnalysis()
m2.fit(X_train, y_train)
m2.score(X_train, y_train)
m2.score(X_test, y_test)

C:\Users\egors\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.6635071090047393

## RNN

In [16]:
from rnn import Encoder
from preprocess import proc
import tensorflow as tf
from tqdm import tqdm
import pandas as pd

encoder = Encoder(300, 300, 100, 1)
encoder.load_weights("encoder.weights")

w2v_dict = dict()
f = open("./glove.840B.300d.txt", "rt", encoding='utf8')
for i in tqdm(range(10000)):
    line = f.readline()
    els = line.split()
    token = els[0]
    try:
        value = tf.convert_to_tensor(list(map(float, els[1:])), dtype=tf.float32)
    except:
        break
    w2v_dict[token] = value
f.close()

def comment_text_to_vec(comment_text):
    p = proc(comment_text, [1, 2])
    tweet = []
    for word in p:
        if word in w2v_dict:
            tweet.append(w2v_dict[word])
    if len(tweet) == 0:
        return None
    tweet = tf.stack(tweet)
    tweet = tf.expand_dims(tweet, axis=1)
    tweet = tf.expand_dims(tweet, axis=3)
    enc_hidden = encoder.initialize_hidden_state()
    for word in tweet:
        out_t, enc_hidden = encoder(word, enc_hidden)
    return enc_hidden[0].numpy()


train = pd.read_csv("./train.csv")[['target', 'comment_text']]
train['toxic'] = 1 * (train['target'] > 0.5)

train_toxic = train[train['toxic'] == 1]
train_not_toxic = train[train['toxic'] == 0].sample(len(train_toxic))
train = pd.concat([train_toxic, train_not_toxic]).sample(len(train_toxic) * 2)


print("Training")
X, y = [], []
for Xrow, yrow in tqdm(list(zip(train['comment_text'], train["toxic"]))[:100]):
    v = comment_text_to_vec(Xrow)
    if v is not None:
        X.append(v)
        y.append(yrow)



100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4391.67it/s]


Training


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [34:31<00:00, 20.72s/it]


In [17]:
import numpy as np
X = np.stack(X)
y = np.array(y)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
m = RandomForestClassifier()
m.fit(X_train, y_train)
print("Train error:", m.score(X_train, y_train))
print("Test error:", m.score(X_test, y_test))

Train error: 0.9863013698630136
Test error: 0.6


In [18]:
pred_rf = m.predict(X_test)

In [19]:
print(confusion_matrix(pred_rf, y_test))
print(classification_report(y_test, pred_rf, digits=3))

[[11  5]
 [ 5  4]]
              precision    recall  f1-score   support

           0      0.688     0.688     0.688        16
           1      0.444     0.444     0.444         9

    accuracy                          0.600        25
   macro avg      0.566     0.566     0.566        25
weighted avg      0.600     0.600     0.600        25



In [ ]:
from sklearn.metrics import f1_score
